In [34]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import matplotlib.pylab as plt

In [35]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [36]:
# Import des questions

In [92]:
answers_grand_debat_LTE_raw = pd.read_csv('raw/large_datasets_ELF/LA_TRANSITION_ECOLOGIQUE.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_FDP_raw = pd.read_csv('raw/large_datasets_ELF/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_ORG_raw = pd.read_csv('raw/large_datasets_ELF/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_DEC_raw = pd.read_csv('raw/large_datasets_ELF/DEMOCRATIE_ET_CITOYENNETE.csv', encoding="utf-8").drop_duplicates(keep=False)

/anaconda3/envs/elfrance/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [95]:
def keep_popular_hashtags_by_question(dataframe):
    allowed_categories_gda = []
    print(dataframe['id_gda'].unique())
    for question in dataframe['id_gda'].unique():
        allowed_categories_tuples = Counter(dataframe[dataframe['id_gda'] == question]['tag_elf']).most_common(20)
        allowed_categories_gda += [(question, element[0], element[1]) for element in allowed_categories_tuples]
    for question in dataframe['id_elf'].unique():    
        allowed_categories_tuples = Counter(dataframe[dataframe['id_elf'] == question]['tag_elf']).most_common(20)
        allowed_categories = [element[0] for element in allowed_categories_tuples]
        dataframe = dataframe[(dataframe['id_elf'] != 'question') | (dataframe['tag_elf'].isin(allowed_categories))]
    return dataframe, allowed_categories_gda

def process_LGD_dataset(raw_dataframe):
    """
    """
    answers_annotees = pd.read_csv('raw/actions.csv', encoding="utf-8").rename(columns={'Contribution': 'reference'})
    print(raw_dataframe.shape)
    raw_dataframe = pd.merge(answers_annotees, raw_dataframe, on='reference')
    if raw_dataframe.shape[0] == 0:
        print("Empty records when matching on annotations")
        return None, None
    
    questions_dictionary = pd.read_csv('raw/match_questions_ELF_LGA.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, questions_dictionary, left_on='Question', right_on='id_gda')
    print(raw_dataframe.shape)

    gda_dictionary = pd.read_csv('raw/matching_gda_id1_id2.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, gda_dictionary, left_on='id_gda', right_on='id1_gda')
    print(raw_dataframe.shape)
    
    raw_dataframe["corpus"] = None
    
    for index, row in raw_dataframe.iterrows():
        raw_dataframe.at[index,'corpus'] = raw_dataframe.loc[index][raw_dataframe.filter(like=row['id2_gda']).columns[0]]

    raw_dataframe = raw_dataframe[['reference', 'id_elf', 'corpus', 'Categorie', 'Annotateur', 'id_gda']]
    
    tags_dictionary = pd.read_csv('raw/matching_tags_lga_elf.csv', sep=';')
    raw_dataframe = pd.merge(raw_dataframe, tags_dictionary, left_on=['id_gda', 'Categorie'], right_on=['id_lga', 'tag_lga'])
    print(raw_dataframe.shape)
    
    # raw_dataframe, allowed_categories_gda = keep_popular_hashtags_by_question(raw_dataframe)
    # print(raw_dataframe.shape)
    
    raw_dataframe = pd.DataFrame({'count' : raw_dataframe.groupby( ['reference', 'tag_elf', 'corpus', 'id_elf'] ).apply(lambda x: x.Annotateur.nunique())}).reset_index()
    print(raw_dataframe.shape)
    
    return raw_dataframe.rename(columns={'reference': 'contribution', 'corpus': 'answer', 'tag_elf': 'target', 'id_elf' : 'question'})
    

In [97]:
answers_grand_debat_GDA_DEC = process_LGD_dataset(answers_grand_debat_DEC_raw)
answers_grand_debat_GDA_FDP = process_LGD_dataset(answers_grand_debat_FDP_raw)
# answers_grand_debat_GDA_ORG, allowed_categories_gda_ORG = process_LGD_dataset(answers_grand_debat_ORG_raw)
answers_grand_debat_GDA_LTE = process_LGD_dataset(answers_grand_debat_LTE_raw)

(73746, 48)
(5915, 55)
(5915, 57)
(2878, 9)
(926, 5)
(126165, 19)
(23033, 26)
(23033, 28)
(17454, 9)
(6804, 5)
(99343, 27)
(20296, 34)
(20296, 36)
(17896, 9)
(5334, 5)


In [98]:
answers_grand_debat_GDA = pd.concat([answers_grand_debat_GDA_FDP, answers_grand_debat_GDA_DEC, answers_grand_debat_GDA_LTE])
answers_grand_debat_GDA['source'] = 'GDA'
answers_grand_debat_GDA.head()

,contribution,target,answer,question,count,source
0,3-10024,"Lutte évasion, fraude, optimisation",Faire payer l'impot à toutes les entreprises s...,FI8,3,GDA
1,3-10024,"Taxer les multinationales, GAFAM",Faire payer l'impot à toutes les entreprises s...,FI8,3,GDA
2,3-10044,Utiliser l'information déjà disponible,Aucune,FI7,2,GDA
3,3-10104,Plus d'information par internet,internet media,FI7,3,GDA
4,3-10104,Plus d'information par les médias,internet media,FI7,3,GDA


In [170]:
answers_grand_debat_ELF = pd.read_csv("raw/elf_tags_sample_v4.csv", sep=";").rename(columns={'corpus': 'answer', 'tag': 'target'})[['contribution', 'target', 'answer', 'question', 'count']]
answers_grand_debat_ELF['source'] = 'ELF'
answers_grand_debat_ELF = answers_grand_debat_ELF[answers_grand_debat_ELF['target'] != 'Autre / Passer']

In [171]:
answers_grand_debat_ELF.target.unique()

array(['Diminution salaires et suppression des avantages des parlementaires, ministres et hauts fonctionnaires',
       'Développement voiture électrique', 'Plus de transparence',
       'Lutte évasion, fraude, optimisation', 'vote obligatoire',
       'Incompétence et inefficacité',
       "Plus d'information sur la dépense des impôts",
       'Lenteur administrative', 'Bureaux de Poste',
       'Famille (enfance, crèches, allocations familiales CAF)',
       'Santé (hôpitaux, médecins)',
       'renforcer les lois (ex : parité H/F, …)',
       'pénaliser / condamner plus fermement',
       "Lutter contre l'évasion et l'optimisation fiscales",
       'Impôt symbolique pour tous', "Plus d'affichage en mairie",
       "Plus d'information par internet", 'Plus de pédagogie',
       'racisme, xénophobie, discriminations religieuses',
       'Plus de contrôle des dépenses (en particulier de la cour des comptes)',
       "Plus d'information sur la collecte des impôts",
       "Plus d'informa

In [177]:
answers_grand_debat = pd.concat([answers_grand_debat_ELF, answers_grand_debat_GDA])
Counter(answers_grand_debat['target']).most_common(20)

[('Tri sélectif, compost, réduction déchets', 1795),
 ('Transports plus verts ou réduits', 1474),
 ('Consommation réduite ou vertueuse (bio, circuit court, local)', 1217),
 ('Plus de transparence', 1138),
 ('Réduction consommation énergie, eau', 941),
 ('Impôt symbolique pour tous', 887),
 ('Plus de tranches IR progressives', 773),
 ('Régime alimentaire', 744),
 ("Plus d'information sur la dépense des impôts", 709),
 ('racisme, xénophobie, discriminations religieuses', 650),
 ('sexisme', 560),
 ('Lutte évasion, fraude, optimisation', 556),
 ('Plus de pédagogie', 552),
 ('Rétablir ISF', 474),
 ('Revoir niches', 455),
 ('Application loi 1905 / Séparation stricte Etat-religions', 440),
 ('Meilleure isolation, changement chauffage/chaudière', 412),
 ('Taxer les riches', 371),
 ('homophobie, transphobie, LGBTphobie', 349),
 ('Cultiver/Jardiner bio, sans pesticide', 326)]

In [178]:
answers_grand_debat_sample = answers_grand_debat.groupby( ['question', 'answer'] ).apply(lambda x: x.target.unique()).reset_index()

In [179]:
answers_grand_debat_sample = answers_grand_debat_sample.rename(columns={0: 'target'})

In [180]:
answers_grand_debat_sample.head()

,question,answer,target
0,DC11,* il faut faire plus de sensibilisation (surto...,[éducation citoyenne de la jeunesse]
1,DC11,- Interdire les sondages en période d'élection...,"[nouveau mode de scrutin (proportionnelle, etc.)]"
2,DC11,- interdire toute possibilité de réélection ap...,"[prise en compte du vote blanc, éducation cito..."
3,DC11,- modernisation des outils de communication et...,"[prise en compte du vote blanc, vote via nouve..."
4,DC11,- vote obligatoire&&- prise en compte du vote ...,"[vote obligatoire, prise en compte du vote bla..."


## Lemmatisation

In [181]:
# spaCy based imports
import spacy
nlp = spacy.load('fr_core_news_sm')

In [ ]:
# Usual imports 
from tqdm import tqdm, tqdm_notebook

# NLP imports
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
import stop_words
import re, sys
from gensim.models import phrases, Phrases

In [ ]:
sw = stop_words.get_stop_words(language='fr')

In [ ]:
answers_grand_debat_sample['tokens'] = answers_grand_debat_sample.answer.map(lambda text: [w for w in text.replace('.', '').replace(',', '').replace('!', '').split() if w])
answers_grand_debat_sample['tokens'] = answers_grand_debat_sample.answer.map(lambda text: word_tokenize(text))
answers_grand_debat_sample['tokens_clean'] =  answers_grand_debat_sample.tokens.map(
                                    lambda tok: [t.lower() for t in re.split(" ", re.sub(r"(\W+|_|\d+)", " ", " ".join(tok))) \
                                                 if t.lower() not in sw and len(t)>1]
                        )
answers_grand_debat_sample['tokens_clean_joined'] =  answers_grand_debat_sample.tokens_clean.map(lambda tok: ' '.join(tok))

In [ ]:
lemmatizer = FrenchLefffLemmatizer()

def lemmatize_answer(answer):
    lemmatized_answer = ''
    nlp_answer = nlp(answer)
    for token in nlp_answer:
        lemmatized_answer += lemmatizer.lemmatize(str(token)) + ' '
    return lemmatized_answer

tqdm.pandas()
answers_grand_debat_sample['lemmatized_answer'] = answers_grand_debat_sample['tokens_clean_joined'].progress_apply(lemmatize_answer)

 79%|███████▉  | 11265/14248 [01:40<00:28, 103.36it/s]

In [ ]:
answers_grand_debat_sample.head()

In [ ]:
answers_grand_debat_final = answers_grand_debat_sample[['question', 'answer', 'lemmatized_answer', 'target']].rename(columns = {'question': 'question_id'})

In [ ]:
answers_grand_debat_final.head()

In [ ]:
questions_ELF

In [ ]:
questions_ELF = pd.read_csv("raw/questions_ELF.csv").rename(columns={'question_elf': 'question'})
answers_grand_debat_final = pd.merge(answers_grand_debat_final, questions_ELF, left_on='question_id', right_on='id_elf')[['theme', 'question_id', 'question', 'answer', 'lemmatized_answer', 'target']]

In [ ]:
answers_grand_debat_final.head()

In [ ]:
answers_grand_debat_final.to_json('preprocessed/reponses_annotees_multilabel.json', orient = "records")